In [1]:
from ultralytics import YOLO
import os
import cv2
import shutil
from sklearn.model_selection import train_test_split
import numpy as np

In [2]:
IMAGES_DIR = './train_dataset/cv_open_dataset/open_img'  # Путь к вашему датасету с изображениями
MASKS_DIR = './train_dataset/cv_open_dataset/open_msk'  # Путь к вашему датасету с масками
OUTPUT_DIR = './datasets/train_data'  # Путь к выходной директории
TRAIN_SIZE = 0.8  # Процент обучающей выборки

In [3]:
os.makedirs(os.path.join(OUTPUT_DIR, 'images/train'), exist_ok=True)
os.makedirs(os.path.join(OUTPUT_DIR, 'images/val'), exist_ok=True)
os.makedirs(os.path.join(OUTPUT_DIR, 'labels/train'), exist_ok=True)
os.makedirs(os.path.join(OUTPUT_DIR, 'labels/val'), exist_ok=True)

In [4]:
image_files = [f for f in os.listdir(IMAGES_DIR) if f.endswith(('.jpg', '.png'))]
mask_files = [f for f in os.listdir(MASKS_DIR) if f.endswith('.png')]
if len(image_files) != len(mask_files):
    print(len(image_files))
    print(len(mask_files))
    print("Количество изображений и масок не совпадает.")
    for mask in mask_files:
        if mask.replace(".png", ".jpg") not in image_files:
            print(mask)

In [5]:
train_images, val_images = train_test_split(image_files, train_size=TRAIN_SIZE, random_state=42)

In [6]:
def convert_mask_to_yolo(mask_path, out):
    # Открываем изображение
    image = cv2.imread(mask_path)

    if image is None:
        print(f"Не удалось открыть изображение: {mask_path}")
        return
    
    height, width = image.shape[:2]

    # Создаем маску для черного цвета
    black_mask = cv2.inRange(image, (0, 0, 0), (50, 50, 50))

    # Создаем новое изображение, где черный цвет остается, а остальные цвета становятся белыми
    new_image = np.ones_like(image) * 255  # Начинаем с белого изображения
    new_image[black_mask > 0] = [0, 0, 0]  # Заменяем черные пиксели

    # Преобразуем в градации серого для нахождения контуров
    gray_image = cv2.cvtColor(new_image, cv2.COLOR_BGR2GRAY)

    # Находим контуры
    contours, _ = cv2.findContours(gray_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Сохраняем контуры в текстовом формате
    output_file_name = os.path.splitext(os.path.basename(mask_path))[0] + '.txt'
    output_file_path = os.path.join(OUTPUT_DIR, out, output_file_name)

    with open(output_file_path, 'w') as f:
        for index, contour in enumerate(contours):
            # Получаем координаты всех точек контура
            contour_points = contour.reshape(-1, 2)
            # Нормализуем координаты
            normalized_points = [(x / width, y / height) for x, y in contour_points]
            points_str = ' '.join(f"{x:.3f} {y:.3f}" for x, y in normalized_points)
            f.write(f"0 {points_str}\n")

In [7]:
# Копирование изображений и масок в соответствующие папки
for img in train_images:
    shutil.copy(os.path.join(IMAGES_DIR, img), os.path.join(OUTPUT_DIR, 'images/train', img))
    mask_name = img.replace('.jpg', '.png')
    convert_mask_to_yolo(os.path.join(MASKS_DIR, mask_name), 'labels/train')

In [8]:
for img in val_images:
    shutil.copy(os.path.join(IMAGES_DIR, img), os.path.join(OUTPUT_DIR, 'images/val', img))
    mask_name = img.replace('.jpg', '.png')
    convert_mask_to_yolo(os.path.join(MASKS_DIR, mask_name), 'labels/val')


In [7]:
data_yaml_content = f"""
train: train_data/images/train
val: train_data/images/val

nc: 1  # Обновите количество классов (1 для загрязнения)
names: ['contaminated']  # Обновите названия классов
"""

with open('data.yaml', 'w') as f:
    f.write(data_yaml_content)

print("Датасет успешно разбит и сохранен в структуре проекта.")

Датасет успешно разбит и сохранен в структуре проекта.


In [10]:
model = YOLO("yolov8n-seg.pt", task='segment')

In [11]:
train_results = model.train(
    data="./data.yaml",  # path to dataset YAML
    epochs=70,  # number of training epochs
    imgsz=640,  # training image size
    device="0",  # device to run on, i.e. device=0 or device=0,1,2,3 or device=cpu
    iou = 0.5, #
    copy_paste = 1.0, #
    # hsv_v = 0.5, #
    mixup= 0.2 #
    # max_det = 100, #
    #mask_ratio=3, #
    # single_cls = True,
    # patience = 20 #обучение останавливается, если метрика не выросла спустя это число эпох
   
)

Ultralytics 8.3.47  Python-3.12.7 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce RTX 4050 Laptop GPU, 6141MiB)
engine\trainer: task=segment, mode=train, model=yolov8n-seg.pt, data=./data.yaml, epochs=70, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=train29, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.5, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True

train: Scanning C:\Users\minim\OneDrive\Документы\Hackathon_nornikel\datasets\train_data\labels\train.cache... 893 images, 214 backgrounds, 0 corrupt: 100%|██████████| 893/893 [00:00<?, ?it/s]
val: Scanning C:\Users\minim\OneDrive\Документы\Hackathon_nornikel\datasets\train_data\labels\val.cache... 392 images, 110 backgrounds, 0 corrupt: 100%|██████████| 392/392 [00:00<?, ?it/s]


Plotting labels to runs\segment\train29\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 66 weight(decay=0.0), 77 weight(decay=0.0005), 76 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs\segment\train29
Starting training for 70 epochs...

      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


       1/70      4.19G      1.308      2.718      1.829      1.289        194        640: 100%|██████████| 56/56 [00:18<00:00,  2.97it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 13/13 [00:03<00:00,  3.78it/s]


                   all        392       1868      0.671      0.421      0.517       0.29      0.667      0.419        0.5      0.243

      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


...

      33/70      3.88G     0.8532      1.631     0.7006      1.065        273        640: 100%|██████████| 56/56 [00:25<00:00,  2.24it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 13/13 [00:04<00:00,  3.04it/s]

                   all        392       1868      0.912      0.825      0.876      0.676      0.918      0.823      0.869      0.616



      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


      34/70      3.93G      0.848      1.608     0.6757      1.049        315        640:  93%|█████████▎| 52/56 [00:18<00:01,  2.85it/s]


KeyboardInterrupt: 

In [4]:
results = model.val(data="./data.yaml")
print(results)

Ultralytics 8.3.47  Python-3.12.7 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce RTX 4050 Laptop GPU, 6141MiB)
YOLOv5s summary (fused): 193 layers, 9,111,923 parameters, 0 gradients, 23.8 GFLOPs


val: Scanning C:\Users\minim\OneDrive\Документы\Hackathon_nornikel\datasets\train_data\labels\val.cache... 392 images, 110 backgrounds, 0 corrupt: 100%|██████████| 392/392 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 25/25 [00:04<00:00,  5.09it/s]


                   all        392       1868      0.903      0.816      0.874      0.646
Speed: 0.3ms preprocess, 5.9ms inference, 0.0ms loss, 2.5ms postprocess per image
Results saved to runs\detect\train22
ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000001AE0A827470>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    

In [4]:
def infer_image(image_path):
    # Загрузка изображения
    image = cv2.imread(image_path)

    # Инференс
    return model(image)



In [5]:
# Функция для создания маски с черным фоном
def create_mask(image_path, results):
    # Загружаем изображение и переводим в градации серого
    image = cv2.imread(image_path)
    height, width = image.shape[:2]

    # Создаем пустую маску с черным фоном
    mask = np.zeros((height, width), dtype=np.uint8)

    # Проходим по результатам и создаем маску
    for result in results:
        masks = result.masks  # Получаем маски из результатов
        if masks is not None:
            for mask_array in masks.data:  # Получаем маски как массивы
                mask_i = mask_array.cpu().numpy()  # Преобразуем маску в numpy массив
                
                # Изменяем размер маски под размер оригинального изображения
                mask_i_resized = cv2.resize(mask_i, (width, height), interpolation=cv2.INTER_LINEAR)
                
                # Накладываем маску на пустую маску (255 для белого)
                mask[mask_i_resized > 0] = 255

    return mask

In [6]:
results = infer_image("./cv_test_dataset/test_img/_09.jpg")
mask_image = create_mask("./cv_test_dataset/test_img/_09.jpg", results)

# Сохраняем маску в формате PNG
mask_output_path = './mask_image.png'  # Укажите путь для сохранения маски
cv2.imwrite(mask_output_path, mask_image)


0: 384x640 3 contaminateds, 43.5ms
Speed: 3.5ms preprocess, 43.5ms inference, 6.5ms postprocess per image at shape (1, 3, 384, 640)


True

In [16]:
!which python

"which" �� ���� ����७��� ��� ���譥�
��������, �ᯮ��塞�� �ணࠬ��� ��� ������ 䠩���.


In [17]:
import os
print(os.environ.get('CUDA_VISIBLE_DEVICES'))

0
